In [3]:
"""

1. Download MERRA2 data at daily interval

2. Resample to ISMIP grid

3. Filter temperature in the ablation zone

4. Save as NetCDF4
    - Temperature 
    - Time

5. Delete original files

"""

# Import libraries
import glob
import os
from datetime import datetime
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4
import pyresample
from pyproj import Transformer
import matplotlib.pyplot as plt

# Define base path
path = '/Users/jryan4/Dropbox (University of Oregon)/research/snowfall/'

# Define path to links
links = pd.read_csv(path + 'data/links/subset_M2T1NXSLV_5.12.4_19800101_20000101_Daily_T2M_Greenland.txt', sep='\t', header=None)
links.rename(columns={0: "link"}, inplace=True)
links['year'] = links['link'].str[119:123]

# Define years
years = np.arange(1980, 2022)

In [4]:
# Read ISMIP data
ismip = xr.open_dataset(path + 'data/masks/1km-ISMIP6.nc')

# Define maximum snowline
snowline_file = netCDF4.Dataset(path + 'data/masks/monthly_bare_ice_2012.nc')
snowline = snowline_file.variables['bare_ice'][1, :, :].filled(np.nan)
max_snowline = (snowline > 0.1)

# Define regions
regions = xr.open_dataset('/Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/temp_albedo_summer_climatologies.nc')

# Define ablation zone coordinates, elevations, and regions
abl_lon = ismip['lon'].values[max_snowline]
abl_lat = ismip['lat'].values[max_snowline]
abl_ele = ismip['SRF'].values[max_snowline]
abl_reg = regions['regions'].values[max_snowline]

# Indices of ablation zone
idx, idy = np.where(max_snowline)

# Define MERRA data
merra_file = xr.open_dataset(path + 'data/merra_sample/MERRA2_200.tavg1_2d_int_Nx.20000101.SUB.nc')

# Meshgrid lat/lons
merra_mesh_lon, merra_mesh_lat = np.meshgrid(merra_file['lon'], merra_file['lat'])

# Define grid definitions for resampling to ISMIP grid
orig_def_regions = pyresample.geometry.GridDefinition(lons=merra_mesh_lon, lats=merra_mesh_lat)
targ_def = pyresample.geometry.GridDefinition(lons=ismip['lon'], lats=ismip['lat'])

In [ ]:
%%capture
# Loop over every link and append a 1D array of ablation zone temperature + time
for year in years:
    
    if os.path.exists(path + 'data/merra_t2m/ablation_t2m_' + str(year) + '.nc'):
        print(f'Skipping...{str(year)}')
    else:
        print(f'Processing...{str(year)}')
        # Make a new DataFrame
        link_year = links[links['year'] == str(year)]

        t = []
        t2m = []
        for j in range(len(link_year)):

            # Index link
            link = '"' + str(link_year.iloc[j].values[0]) + '"'

            # Download MERRA2 using WGET
            !wget --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --auth-no-challenge=on --keep-session-cookies --no-check-certificate --content-disposition $link --directory-prefix=tmp_t -nd

            # Import temporary file
            merra = xr.open_dataset(sorted(glob.glob(path + 'repo/tmp_t/*.nc'))[0])
            
            # Clean up temporary files
            files = glob.glob(path + 'repo/tmp_t/*.nc')
            for f in files:
                os.remove(f)

            # Determine nearest (w.r.t. great circle distance) neighbour in the grid.
            t2m_resample = pyresample.kd_tree.resample_nearest(source_geo_def=orig_def_regions, 
                                                         target_geo_def=targ_def, 
                                                         data=np.mean(merra['T2M'].values, axis=0), 
                                                         radius_of_influence=50000)

            # Append to list
            t2m.append(t2m_resample[max_snowline])
            t.append(merra['time'].values[0])

        # Save as NetCDF
        ds_data = xr.Dataset(
        data_vars={
            "t2m": (("time", "x"), np.array(t2m).astype('float32')),
        },

        coords={
            "time": pd.DatetimeIndex(t, freq='D'),
            "longitude": (('x',), np.array(abl_lon)),
            "latitude": (('x',), np.array(abl_lat)),    
        },

        attrs={
            "Produced": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Units": 'K',
            "Author":'Johnny Ryan', 
            "Email":'jryan4@uoregon.edu'
        },
        )

        # Save
        ds_data.to_netcdf(path + 'data/merra_t2m/ablation_t2m_' + str(year) + '.nc')
        